# El problema.



In [ ]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
import os
import time
import random
import math
from google.colab import files

La intención de este proyecto es ayudar a un grupo de cuatro amigos a tomar la mejor decisión al momento de comprar su equipo de cómputo para su trabajo. Cada uno de estas personas tienen preferencias distintas por las marcas que podrían comprar, por lo que se va a definir una lista con su nombre y la marca que prefieren, así como también la especificación obligatoria para la compra. 

In [ ]:
people = [('Bart', 'Lenovo'),
          ('Lisa', 'HP'),
          ('Homer', 'Dell'),
          ('Marge', 'Asus')]

In [ ]:
esp_obl = 'SSD'

Existen muchas laptops con diferentes especificaciones y distintos precios.

Podemos obtener información de las disponibles a partir del archivo `Laptop_data.csv`.

In [ ]:
def load_lap(path):
    lap = {}
    with open(path) as f:
        f.readline()
        for line in f:
            cols = line.strip().split(',')
            Company = cols[1]
            Cpu = cols[6]
            Gpu = cols[7]
            lap[Company] = (
                Cpu,
                Gpu
            )
    return lap

In [ ]:
lap = load_lap('laptop_datas.csv')

Se puede obtener la información relevante de las laptops a partir de `Laptop_data.csv`.

In [ ]:
def load_laptop(path):
    laptop = {}
    with open(path) as f:
        for line in f:
          Company, Disk, ScreenResolution, GHz, RamGB, MemoryGB, ExtraGB, Price = line.strip().split(',')
          laptop.setdefault((Company, Disk), [])
          laptop[(Company, Disk)].append((int(float(ScreenResolution)), float(GHz), int(float(RamGB)), 
                                          int(float(MemoryGB)), int(float(ExtraGB)), int(float(Price))))
    return laptop

In [ ]:
laptop = load_laptop('laptop2.csv')

Podemos consultar las posibles compras de para un miembro del equipo (en este ejemplo el primero en la lista, que es Bart) de la siguiente forma:

In [ ]:
laptop[(
    people[0][1],
    esp_obl,
)]

[(1920, 2.5, 8, 128, 1024, 638),
 (1366, 2.0, 4, 128, 0, 235),
 (1920, 2.7, 8, 256, 0, 382),
 (1920, 2.5, 4, 128, 0, 255),
 (1920, 2.7, 8, 256, 0, 946),
 (1920, 2.5, 8, 256, 0, 894),
 (1920, 2.5, 8, 256, 0, 402),
 (1920, 1.6, 8, 256, 0, 713),
 (1920, 2.8, 8, 256, 0, 555),
 (1366, 2.5, 4, 128, 0, 318),
 (1920, 2.4, 4, 256, 0, 402),
 (1920, 2.8, 16, 256, 0, 689),
 (1920, 2.5, 8, 128, 1024, 519),
 (3840, 1.8, 16, 512, 0, 1182),
 (1920, 2.5, 8, 256, 0, 498),
 (2560, 2.7, 16, 1024, 0, 1805),
 (1600, 1.6, 8, 256, 0, 446),
 (1366, 2.3, 4, 128, 0, 315),
 (1920, 2.8, 16, 512, 1024, 1118),
 (1920, 2.8, 16, 512, 0, 893),
 (1366, 2.5, 8, 256, 0, 381),
 (1920, 2.5, 8, 256, 0, 351),
 (1920, 1.6, 8, 256, 0, 574),
 (1920, 2.8, 16, 256, 1024, 958),
 (1366, 2.0, 8, 128, 0, 376),
 (1920, 2.7, 8, 256, 0, 689),
 (1920, 1.8, 8, 512, 0, 1182),
 (1920, 2.8, 8, 512, 0, 1086),
 (1920, 2.0, 4, 256, 0, 283),
 (1920, 1.8, 8, 256, 0, 562),
 (2560, 2.6, 16, 512, 0, 1597),
 (1600, 2.7, 6, 128, 1024, 549),
 (1920, 2.5

Vamos a considerar una estructura particular para las posibles soluciones al problema. Una representación usual es que las soluciones sean listas de números. En nuestro caso, cada número puede representar la compra, de tal manera que el tamaño de la solución es igual a la cantidad de personas.

Por ejemplo,
```
[1,3,7,6]
```

Nos representa una solución donde:
- la persona con índice `0` compra la laptop con índice `1`,
- la persona con índice `1` compra la laptop con índice `3`,
- la persona con índice `2` compra la laptop con índice `7`,
- la persona con índice `3` compra la laptop con índice `6`.

El índice de las personas hace referencia a la lista `people`, si dicha persona prefiere la marca `A` y la RAM `B`, entonces el índice de compra hace referencia a la lista `laptop[(A,B)]`.

Todo esto puede sonar confuso, lo conveniente es elegir una representación lo suficientemente simple para que nuestros programas encuentren buenas soluciones, pero lo suficientemente complejo como para entender soluciones como personas.

La siguiente función nos permite tomar un valor de solución e imprimir la información.

In [ ]:
def print_buy(s):
    for i in range(len(s)):
        name = people[i][0]
        Company = people[i][1]
        buy = laptop[(Company, esp_obl)][s[i]]
        print(
            name,
            Company,
            lap[people[i][1]][1],
            buy[0],buy[1],buy[2],buy[3],buy[4],buy[5],
        )

In [ ]:
print_buy([70,50,80,55])

Bart Lenovo Intel HD Graphics 1920 2.5 8 180 0 872
Lisa HP AMD Radeon R5 M330 1920 2.8 8 128 1024 702
Homer Dell AMD Radeon R5 M430 1920 1.8 8 128 1024 625
Marge Asus Intel HD Graphics 1920 2.5 8 128 1024 1054


Hasta ahora, hemos modelado este problema de forma computacional y tenemos codificaciones prácticas para razonar sobre personas, laptops y especificaciones.
Sin embargo, podemos ver que la solución de ejemplo nos dice que Homer debería comprar una laptop con un precio muy grande y eso no es una buena solución.

# Función de costo



Vamos a definir una *función de costo*, de tal manera que el problema se va a reducir a encontrar un conjunto de entradas (compra de laptop en este caso) que minimice la función de costo, es decir, la que tenga el costo más bajo.

Esta función de costo debe recibir una posible solución y darnos un valor numérico que nos indique qué tan mala es.

Suele ser dificil determinar qué hace que una solución sea buena o mala cuando se involucran varios factores. Consideremos algunos candidatos para este problema:

- Resolución: Resolución de la pantalla.
- GHz: Cantidad de gigahercios en el procesador.
- RAM: Cantidad de memoria RAM. 
- Almacenamiento: Cantidad de almacenamiento ya sea SSD o HDD.
- Precio: El precio total de compra.
- Almacenamiento extra: Cantidad de almacenamiento extra en un disco diferente.


Podemos imaginarnos otros factores que tomar en cuenta para nuestro problema particular.

Una vez que determinamos el conjunto de factores que afectan nuestra noción de *costo*, debemos determinar cómo combinarlas en un número.

In [ ]:
def buy_cost(s):
    # contamos el precio total de cada posible compra
    total_price = 0
    
    # nos interesa conocer el presupuesto límite de compra
    budget = 0

    #Resto de especificaciones
    resolution = 0
    ghz = 0
    ram = 0
    storage = 0
    xtrastor = 0

    
    for i in range(len(s)):
        name = people[i][0]
        Company = people[i][1]
        buy = laptop[(Company, esp_obl)][s[i]]
        
        total_price += buy[5]/s.count(s[i]) # Precio total va a ser la suma de los precios individuales. 
  
  # Se establece un presupuesto máximo.
        budget += (3000 if  s.count(s[i]) < buy[5] else 0)

  # Resto de especificaciones
        resolution += buy[0]
        ghz += buy[1]
        ram += buy[2]
        storage += buy[3]
        xtrastor += buy[4]

# El costo-beneficio total es la suma de la ponderación que se le da a cada especificación.
  
    return 0.15*total_price - 0.02*resolution - 0.07*ghz - 0.25*ram - 0.25*storage - 0.01*xtrastor + 0.25*budget
    

A continuación se muestra que tan bueno/malo es el ejemplo con los criterios establecidos:

In [ ]:
buy_cost([3,15,20,55])

2987.581

# Buscar todas las soluciones

Ahora se obtendrá la cantidad de combinaciones de compra que se pueden realizar para después poder obtener la de menor costo.

In [ ]:
print(f"Hay {len(people)} personas en el equipo:")
total_solutions = 1
for p in people:
    name = p[0]
    Company = p[1]
    buy3 = len(laptop[(p[1],esp_obl)])
    total_solutions *= buy3
    print(f"- {name} quiere una laptop {Company} con SSD, tiene {buy3} opciones de compra;")
print(f"Por lo que hay un total de {total_solutions} posibles soluciones que analizar!\n\n")

if total_solutions > 1e9:
    print("¡A la bestia!, son un chorro de soluciones posbiles.")
else:
    print("Meh, no es tan dificil como parece encontrar la mejor solución.")

Hay 4 personas en el equipo:
- Bart quiere una laptop Lenovo con SSD, tiene 199 opciones de compra;
- Lisa quiere una laptop HP con SSD, tiene 164 opciones de compra;
- Homer quiere una laptop Dell con SSD, tiene 196 opciones de compra;
- Marge quiere una laptop Asus con SSD, tiene 100 opciones de compra;
Por lo que hay un total de 639665600 posibles soluciones que analizar!


Meh, no es tan dificil como parece encontrar la mejor solución.


# Aleatoria

Se va a intentar encontrar un buen resultado haciendo una búsqueda aleatoria en el espacio de soluciones.



Consideremos la función `solve_randomly` que toma dos parámetros:
1. El *dominio* que consiste en una secuencia de tuplas `(min, max)` que establecen el valor mínimo y máximo que pueden tomar las entradas de las soluciones (de esta forma, codificamos el espacio de posibles soluciones de manera sucinta)
2. Una función de *costo*, que toma una posible solución y nos regresa un valor numérico que queremos minimizar.

Es importante observar que la cantidad de elementos en el dominio es igual a la cantidad de elementos en una solución.

Vamos a generar de forma aleatoria soluciones y regresar aquella con el costo mas pequeño.

In [ ]:
def random_solution(domain):
    return [
        random.randint(r[0], r[1])
        for r in domain
    ]

In [ ]:
def solve_randomly(domain, cost_of, repeats = 1000):
    best_cost = float('inf')
    best_sol = None
    
    for _ in range(repeats):
        s = random_solution(domain)
        c = cost_of(s)
        if c < best_cost:
            best_cost = c
            best_sol = s
    
    return s

In [ ]:
domain = [(0,9)] * len(people)

In [ ]:
def test_randomly(repeats = 1000):
    s = solve_randomly(
        domain,
        buy_cost,
        repeats
    )
    print_buy(s)
    print(f"\nCon costo {buy_cost(s)}")

In [ ]:
test_randomly()

Bart Lenovo Intel HD Graphics 1920 2.5 8 128 1024 638
Lisa HP AMD Radeon R5 M330 1920 1.6 8 128 1024 624
Homer Dell AMD Radeon R5 M430 1920 2.5 4 256 0 408
Marge Asus Intel HD Graphics 1920 2.4 6 256 0 368

Con costo 2858.09


In [ ]:
%timeit -n 1 -r 1 test_randomly()

Bart Lenovo Intel HD Graphics 1920 2.8 8 256 0 555
Lisa HP AMD Radeon R5 M330 1920 2.5 8 256 0 441
Homer Dell AMD Radeon R5 M430 1920 2.0 4 256 0 318
Marge Asus Intel HD Graphics 1920 2.7 8 256 0 762

Con costo 2894.1
25.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_randomly(int(1e4))

Bart Lenovo Intel HD Graphics 1366 2.0 4 128 0 235
Lisa HP AMD Radeon R5 M330 1920 1.6 8 128 1024 624
Homer Dell AMD Radeon R5 M430 1920 1.6 8 256 0 523
Marge Asus Intel HD Graphics 1920 2.0 4 256 0 265

Con costo 2836.686
136 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_randomly(int(2e4))

Bart Lenovo Intel HD Graphics 1366 2.5 4 128 0 318
Lisa HP AMD Radeon R5 M330 1920 1.6 8 128 1024 624
Homer Dell AMD Radeon R5 M430 1920 2.7 8 256 0 476
Marge Asus Intel HD Graphics 1920 3.0 8 256 1024 830

Con costo 2876.564
247 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Descenso de colinas


Intentar obtener un buen resultado generando soluciones aleatorias es una ineficiente y pésima estrategia en este caso porque hay mas de medio millon de posibilidades. 

Un problema obvio de el método es que no aprovecha la información de las mejores soluciones que ha generado para generar otras buenas soluciones.

En nuestro problema particular, una solución con bajo costo es probablemente similar a otras soluciones con bajo costo.

Vamos a incorporar esta idea implementando en Python un método alternativo llamado *descenso de colinas*. Comenzamos con una solución aleatoria y buscamos en la *vecindad* de la solución por aquellas que mejoran el costo.


Detendremos la búsqueda hasta llegar a una solución cuya vecindad no mejora el costo.

In [ ]:
def neighbors_of(s, domain):
    neighbors = []
    for i in range(len(domain)):
        if s[i] > domain[i][0]:
            neighbors.append(s[0:i] + [s[i] - 1] + s[i+1:])
        if s[i] < domain[i][1]:
            neighbors.append(s[0:i] + [s[i] + 1] + s[i+1:])
    return neighbors

Este criterio de vecindad corresponde a todas las soluciones que están a distancia 1 (de acuerdo a Hamming).

Veamos la lista de vecinos de nuestra solución de ejemplo:

In [ ]:
neighbors_of([70,50,80,55], domain)

[[69, 50, 80, 55], [70, 49, 80, 55], [70, 50, 79, 55], [70, 50, 80, 54]]

In [ ]:
def solve_hillclimbing(domain, cost_of):
    s = random_solution(domain)
    
    while True:
        neighbors = neighbors_of(s, domain)
        cost = cost_of(s)
        best_neighbor = min(neighbors, key=cost_of)
        neighbor_cost = cost_of(best_neighbor)
        
        if cost < neighbor_cost:
            return s
        
        s = best_neighbor

In [ ]:
def test_hillclimbing():
    s = solve_hillclimbing(
        domain,
        buy_cost,
    )
    print_buy(s)
    print(f"\nCon costo {buy_cost(s)}")

In [ ]:
%timeit -n 1 -r 1 test_hillclimbing()

Bart Lenovo Intel HD Graphics 1920 2.5 4 128 0 255
Lisa HP AMD Radeon R5 M330 1920 2.5 4 256 0 254
Homer Dell AMD Radeon R5 M430 1920 1.6 8 256 0 523
Marge Asus Intel HD Graphics 1920 2.4 6 256 0 368

Con costo 2723.07
6.62 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
test_hillclimbing()

Bart Lenovo Intel HD Graphics 1920 2.7 8 256 0 382
Lisa HP AMD Radeon R5 M330 1920 2.5 8 256 0 367
Homer Dell AMD Radeon R5 M430 1920 2.8 16 256 1024 958
Marge Asus Intel HD Graphics 1920 2.4 6 256 0 368

Con costo 2781.732


# Recocido simulado

Un problema del método de descenso de colinas es que una vez que se llega a un mínimo local, se deja de buscar. Sin embargo, este mínimo local puede corresponder a una mala solución.

Podemos perturbar la búsqueda con un método llamado *recocido simulado*.



Este método consiste en partir de un estado caliente, en donde es probable seguir un vecino con peor costo. Poco a poco, la temperatura del sistema baja, de tal forma que cada vez es menos probable elegir un peor vecino. Eventualmente, la temperatura es suficientemente baja como para detener la búsqueda.

Esto nos permite introducir una probabilidad variable para evitar algunos mínimos locales.

La estrategia es la siguiente: comenzamos con una solución aleatoria y una temperatura alta, tomamos un vecino aleatorio y analizamos los casos:
1. Si el vecino tiene menor costo $c'$ que la solución actual con costo $c$, elegimos al vecino como nueva solución
2. Si no es el caso, elegimos al vecino como nueva solución con probabilidad $$\mathrm{e}^{(c-c')/T}$$

In [ ]:
def solve_annealing(domain, cost_of, Ti=10000.0, Tf=0.1, alpha=0.95):
    solution = random_solution(domain)
    cost = cost_of(solution)
    T = Ti
    while T > Tf:
        neighbor = random.choice(neighbors_of(solution, domain))
        neighbor_cost = cost_of(neighbor)
        diff = cost - neighbor_cost
        if diff > 0 or random.random() < (math.exp(diff / T)):
            solution = neighbor
            cost = neighbor_cost
        T = alpha*T
    
    return solution

In [ ]:
def test_annealing(Ti=100000.0, Tf=0.1, alpha=0.95, beta=0):
    s = solve_annealing(
        domain,
        buy_cost,
        Ti, Tf, alpha,
    )
    print_buy(s)
    print(f"\nCon costo {buy_cost(s)}")

In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=10000.0, Tf=0.1, alpha=0.95)

Bart Lenovo Intel HD Graphics 1920 2.5 8 256 0 402
Lisa HP AMD Radeon R5 M330 1920 2.5 8 256 0 441
Homer Dell AMD Radeon R5 M430 3200 1.8 16 512 0 1194
Marge Asus Intel HD Graphics 1920 2.8 16 128 1024 926

Con costo 2621.0005
14.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=10000.0, Tf=0.1, alpha=0.99)

Bart Lenovo Intel HD Graphics 1920 2.8 8 256 0 555
Lisa HP AMD Radeon R5 M330 1920 1.6 8 128 1024 624
Homer Dell AMD Radeon R5 M430 1920 2.8 16 256 1024 958
Marge Asus Intel HD Graphics 1920 1.8 16 512 0 955

Con costo 2775.39
38.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=100000.0, Tf=0.1, alpha=0.99)

Bart Lenovo Intel HD Graphics 1920 2.5 8 256 0 402
Lisa HP AMD Radeon R5 M330 1920 1.8 8 512 0 705
Homer Dell AMD Radeon R5 M430 1920 1.6 8 256 0 511
Marge Asus Intel HD Graphics 1920 2.8 16 128 1024 926

Con costo 2728.351
55.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.99)

Bart Lenovo Intel HD Graphics 1920 2.5 8 256 0 402
Lisa HP AMD Radeon R5 M330 1920 2.5 4 256 0 254
Homer Dell AMD Radeon R5 M430 3200 1.8 16 512 0 1194
Marge Asus Intel HD Graphics 1920 2.0 4 256 0 265

Con costo 2650.809
49 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -n 1 -r 1 test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.999)

Bart Lenovo Intel HD Graphics 1920 2.7 8 256 0 946
Lisa HP AMD Radeon R5 M330 1920 1.8 8 512 0 705
Homer Dell AMD Radeon R5 M430 1920 2.0 4 256 0 318
Marge Asus Intel HD Graphics 1920 1.6 8 256 0 601

Con costo 2679.133
216 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Algoritmos genéticos



Ahora vamos a considerar otro método, que al igual que el recocido simulado está inspirado en la naturaleza, pero no en la física, si no en la biología.

Primero se crea un conjunto de soluciones aleatorias, conocidas como *la población*. En cada paso del método, la función de costo para cada solución en la población es calculada, esto nos permite ordenar las soluciones de mejor a peor.

Posteriormente, una nueva población es generada a partir de la actual: Las mejores soluciones de la actual se conservan tal cuál (*elitismo*). El resto de la población, va a ser modificada para obtener la nueva población.

Hay dos formas en que las soluciones pueden ser modificadas. La mas simple es llamada *mutación* y consiste en un pequeño y simple cambio aleatorio a la solución actual. Esto sería similar a elegir un vecino de forma aleatoria.

Otra manera de modificar las soluciones es llamado *cruza*, consiste en tomar dos soluciones de las mejores y combinarlas de alguna manera. Una forma simple de combinarlas es tomar una cantidad aleatoria de elementos de una buena solución y acompletar los elementos que faltan con otra buena solución.

Una vez que se obtiene la nueva población, el proceso continúa.

In [ ]:
def mutate(s, domain):
    return random.choice(neighbors_of(s, domain))

In [ ]:
def crossover(s1, s2):
    i = random.randint(1, len(s1)-2)
    return s1[0:i] + s2[i:]

In [ ]:
def solve_evolving(domain, cost_of, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
    pop = [random_solution(domain) for _ in range(pop_size)]
    top_elite = int(elite * pop_size)
    
    for epoch in range(epochs):
        pop.sort(key=cost_of)
        best = pop[0:top_elite]
        while len(best) < pop_size:
            if random.random() < mut_prob:
                best.append(mutate(
                    best[random.randint(0, top_elite-1)],
                    domain
                ))
            else:
                best.append(crossover(
                    best[random.randint(0, top_elite-1)],
                    best[random.randint(0, top_elite-1)],
                ))
        pop = best
    pop.sort(key=cost_of)
    return pop[0]

In [ ]:
def test_evolving(pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
    s = solve_evolving(
        domain,
        buy_cost,
        pop_size, mut_prob,
        elite, epochs,
    )
    print_buy(s)
    print(f"\nCon costo {buy_cost(s)}")

In [ ]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 100)

Bart Lenovo Intel HD Graphics 1920 2.7 8 256 0 946
Lisa HP AMD Radeon R5 M330 1920 1.8 8 512 0 705
Homer Dell AMD Radeon R5 M430 1920 1.6 8 256 0 511
Marge Asus Intel HD Graphics 1920 2.4 6 256 0 368

Con costo 2681.605
102 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 200)

Bart Lenovo Intel HD Graphics 1920 2.5 8 128 1024 638
Lisa HP AMD Radeon R5 M330 1920 2.5 8 256 0 367
Homer Dell AMD Radeon R5 M430 1920 2.0 4 256 0 318
Marge Asus Intel HD Graphics 1920 1.8 16 512 0 955

Con costo 2623.969
173 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%timeit -r 1 -n 1 test_evolving(20, 0.2, 0.2, 100)

Bart Lenovo Intel HD Graphics 1920 2.7 8 256 0 382
Lisa HP AMD Radeon R5 M330 1920 1.6 8 256 0 561
Homer Dell AMD Radeon R5 M430 1920 2.0 4 256 0 318
Marge Asus Intel HD Graphics 1366 2.5 4 256 0 374

Con costo 2708.4139999999998
54.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# La tarea

1. Explora los parámetros y la función costo de este problema, intenta encontrar mejores valores para resolver el problema.
2. Elige un problema distinto, y modela las posibles soluciones del problema para utilizar los métodos discutidos en esta libreta: **búsqueda de solución aleatoria, búsqueda de solución por descenso de colinas, búsqueda de solución por recocido simulado, búsqueda de solución por algoritmos genéticos.**
3. Compara los resultados de las soluciones de los cuatro métodos anteriores.
4. ¿Es posible resolver el problema que planteas analizando todas las posibles soluciones? Justifica tu respuesta.

1. Notebook disponible [aquí](https://colab.research.google.com/drive/1isqhY8fubL01gnAfI18cSq4TtTBvZ6sv?usp=sharing).

2. Todo lo anterior. 

3. De los métodos utilizados, el que despliega un mejor resultado es el de algoritmos genéticos, seguidos del recocido simulado, descenso de colinas y por último el método aleatorio. 

4. Sí es posible resolver el problema de compra de laptops para un equipo de personas con estas posibles soluciones si se considera que lo que menos se quiere es hacer algun cambio de partes después de la compra, por ejemplo agregar memoria ram o hacer un aumento de almacenamiento. 
La forma en la que este problema puede ser resuelto es ponderando la importancia que se le da a cada especificación de la laptop, en este caso sería que se quiere una laptop con disco tipo ssd, con la mayor resolución de pantalla, mayor GHz, mayor RAM, mayor almacenamiento y el menor precio posible, considerando un presupuesto establecido. 